In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Introduction


This is the for 

* module `fieldnn.basicnn.expander` module
* module `fieldnn.configfn.expanderfn` module

# Batch

In [2]:
import os

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P@PatEcInfo-InfoGrn',  'P-EC-PNSect@AllText-TknzGrn']

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

In [3]:
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')

4 <---- dataset
1 <---- dataset


In [4]:
for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

B-P@PatEcInfo-InfoGrn_wgt torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect@AllText-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [5]:
batch_y

tensor([1, 0, 1, 1])

In [6]:
# batch_rfg
# B - P - EC -PNSect@AllText-TknzGrn_tknidx 
# [ 4,    87,      15,     283]
# batch_rfg['B-P-EC-PNSect@AllText-TknzGrn_tknidx']#[:, 0, 0, :]

## Analyze One RecFldGrn

**Understand Holder**


Introduction to Holder.

You can treat the holder as the position holder.

One holder is always with a info (tensor vectors).

We can have two types of holder:

(a) the grain_idx. **for example: **
    
```python
holder: [[123,   333,  123,  333], [ 56,  24,  23, 0],[ 24,  23, 0]]
info:   [[v123, v333, v123, v333], [v56, v24, v23, 0],[v24, v23, 0]]  
```

(b) the length of chidren for the focal element. **for example: after we process the above tensor to the sentence-level tensor. we will have this:**
       
```python
holder: [4,   3,  2]
info:   [s1, s2, s3]  # (here s1 is a sentence-level vector from [v123, v333, v123, v333]
```

**Relationships between leng_mask, leng, holder, and info**

1. Current Layer:

```python
leng_mask = holder == 0 # 
# for leng_mask: if with real value, then False, if with padding, then True
# in the real mask, we need to unsqueeze() the last dim. 
leng = (leng_mask == 0).sum(-1) # in leng_mask: 0 means a real position
``` 

2. Transfer

```python # 
holder = leng # (for next iteration)
```


3. Next Layer
```python
leng_mask = holder == 0
leng = (leng_mask == 0).sum(-1) # in leng_mask: 0 means a real position
```


In [7]:
[i for i in batch_rfg]

['B-P@PatEcInfo-InfoGrn_wgt',
 'B-P@PatEcInfo-InfoGrn_tknidx',
 'B-P@PatEcInfo-InfoGrn_fldidx',
 'B-P-EC-PNSect@AllText-TknzGrn_wgt',
 'B-P-EC-PNSect@AllText-TknzGrn_tknidx',
 'B-P-EC-PNSect@AllText-TknzGrn_fldidx']

In [8]:
recfldgrn_list

['P@PatEcInfo-InfoGrn', 'P-EC-PNSect@AllText-TknzGrn']

In [9]:
import torch

recfldgrn = 'P-EC-PNSect@AllText-TknzGrn'
full_recfldgrn = f'B-{recfldgrn}' # 'P-EC-PNSect@AllText-TknzGrn'

full_recfldgrn_tknidx = f'{full_recfldgrn}_tknidx'
info_idx = batch_rfg[full_recfldgrn_tknidx]
print(info_idx.shape)
holder = info_idx.long()

torch.Size([4, 23, 14, 221])


In [10]:
print(full_recfldgrn, '<-- full_recfldgrn')

recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn

print(recfldgrn, '<---- recfldgrn')
print(prefix_ids, '<---- prefix_ids')
print(rec, '<---- rec')
print(fld, '<---- fld')
print(grn, '<---- grn')

B-P-EC-PNSect@AllText-TknzGrn <-- full_recfldgrn
PNSect@AllText-TknzGrn <---- recfldgrn
['B', 'P', 'EC'] <---- prefix_ids
PNSect <---- rec
AllText <---- fld
TknzGrn <---- grn


# BasicNN Name

Expander try to convert GRN_idx (or GRN_wgt) to embedding vectors.

It is from idx to vector.

It is a type of order-increase. 


Here the core include: CateEmbedding, NumeEmbedding, and LLMEmbed. 



## CateEmbedding

### Module

In [11]:
import os
import torch
import numpy as np

class CateEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 vocab_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(CateEmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(vocab_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, holder):
        # info is the grain
        info = self.embedding(holder)
        return info

### Config

In [12]:
### Main Config
### others will change with each recfldgrn

#######################
embed_size = 128
init = 'random'
#######################

In [13]:
def get_cateembed_para(embed_size, vocab_size, init = 'random'):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'vocab_size': vocab_size}
    return embed_para

### Usage 

In [14]:
import torch

recfldgrn = 'P-EC-PNSect@AllText-TknzGrn'
full_recfldgrn = f'B-{recfldgrn}'

full_recfldgrn_tknidx = f'{full_recfldgrn}_tknidx'
info_idx = batch_rfg[full_recfldgrn_tknidx]
print(info_idx.shape)
holder = info_idx.long()

torch.Size([4, 23, 14, 221])


In [15]:
print(full_recfldgrn, '<-- full_recfldgrn')

recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn

print(recfldgrn, '<---- recfldgrn')
print(prefix_ids, '<---- prefix_ids')
print(rec, '<---- rec')
print(fld, '<---- fld')
print(grn, '<---- grn')

B-P-EC-PNSect@AllText-TknzGrn <-- full_recfldgrn
PNSect@AllText-TknzGrn <---- recfldgrn
['B', 'P', 'EC'] <---- prefix_ids
PNSect <---- rec
AllText <---- fld
TknzGrn <---- grn


In [16]:
import pandas as pd
# (2) get vocab information, in the `recfldgrn` task, we have already created the FldGrnInfo
fldgrn_folder = 'data/ProcData/FldGrnInfo/'
fullfldgrn_file = os.path.join(fldgrn_folder, full_recfldgrn[2:] + '.p')
print(fullfldgrn_file)
Info = pd.read_pickle(fullfldgrn_file)
Info

data/ProcData/FldGrnInfo/P-EC-PNSect@AllText-TknzGrn.p


RecLevel                                                          PNSect
RecLevelID                                                      PNSectID
SynFld                                                           AllText
GrnName                                                          TknzGrn
Rec2FldList_Dict       {'PNSect': ['SectName'], 'PNSectSent': ['Sente...
prefix_ids                                                   [PID, ECID]
focal_ids                                                     [PNSectID]
rec_folder                                       data/ProcData/RecFolder
recfldgrn                                         PNSect@AllText-TknzGrn
prefix_recfldgrn                             P-EC-PNSect@AllText-TknzGrn
grain_tkn_fn_string    def grain_tkn_fn(df_input):\n    # (0) Set up....
EmbedDict              {'tkn': {'embed_type': 'LLMEmbed', 'recfldgrn_...
use_wgt                                                            False
dtype: object

In [17]:
for sfx, para in Info['EmbedDict'].items():
    if para['embed_type'] == 'CateEmbed':
        print(sfx)
        sfx, para
        break

fld


In [18]:
para

{'embed_type': 'CateEmbed',
 'recfldgrn_sfx': 'PNSect@AllText-TknzGrn_fld',
 'vocab_size': 3,
 'Vocab': {'idx2v': {0: '_padding', 1: 'Sentence', 2: 'SectName'},
  'v2idx': {'_padding': 0, 'Sentence': 1, 'SectName': 2},
  'v2freq': {'_padding': 0, 'Sentence': 18002, 'SectName': 2368}},
 'tknz': None}

In [19]:
# # (3) Query the v2idx_tokenizer information from the df_FieldGrainInfo
# # here v2idx could be a vocab dictionary for categorical data, or tokenizer for the texutal data.
# vocab_tokenizer = Info['VocabDict']['tkn']['v2idx']

# # (4) get the vocab_size for the v2idx_tokenizer. 
# # ps. even tokenizer (from huggingface) can have use len().
# vocab_size = len(vocab_tokenizer)
# print(vocab_size)
vocab_size = para['vocab_size']
vocab_size


3

**get configuration**


Here we want to get the embedding configuration.

In [20]:
embed_para = get_cateembed_para(embed_size, vocab_size, init)
print(embed_para)

{'embedding_size': 128, 'init': 'random', 'vocab_size': 3}


**init NN model**

In [21]:
NN = CateEmbeddingLayer(**embed_para)
NN

CateEmbeddingLayer(
  (embedding): Embedding(3, 128, padding_idx=0)
)

**prepare input**

In [22]:
full_recfldgrn

'B-P-EC-PNSect@AllText-TknzGrn'

In [23]:
# (1) get the info_raw from batch_rfg

full_recfldgrn_sfx = full_recfldgrn + f'_{sfx}idx'
info_raw = batch_rfg[full_recfldgrn_sfx]
print(info_raw.shape, f'<---- info_raw for full_recfldgrn {sfx}: {full_recfldgrn_sfx}' )


torch.Size([4, 23, 14, 221]) <---- info_raw for full_recfldgrn fld: B-P-EC-PNSect@AllText-TknzGrn_fldidx


In [24]:
full_recfldgrn_wgt = full_recfldgrn + f'_wgt'
info_wgt = batch_rfg[full_recfldgrn_wgt]
print(info_wgt.shape, f'<---- full_recfldgrn wgt: {full_recfldgrn_wgt}' )

torch.Size([4, 23, 14, 221]) <---- full_recfldgrn wgt: B-P-EC-PNSect@AllText-TknzGrn_wgt


In [25]:

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)

# here you may wonder what is holder.
# You can treat the holder as the position holder.
# One holder is always with a info (tensor vectors).

# We can have two types of holder:
# (a) the grain_idx 
#     for example: 
#        holder: [[123,   333,  123,  333], [ 56,  24,  23, 0],[ 24,  23, 0]]
#        info:   [[v123, v333, v123, v333], [v56, v24, v23, 0],[v24, v23, 0]]  
#
# (b) the length of chidren for the focal element.
#     for example: after we process the above tensor to the sentence-level tensor.
#                  we will have this:
#        holder: [4,   3,  2]
#        info:   [s1, s2, s3]  (here s1 is a sentence-level vector from [v123, v333, v123, v333]
# 

# depending on the input type, we will create holder, and holder_wgt accordingly.

holder = info_raw.long()
holder_wgt = info_raw.float() # torch.FloatTensor(info_raw)
    
print(holder.shape, '<------ holder shape')
print(holder_wgt.shape, '<------ holder wgt information')


torch.Size([4, 23, 14, 221]) <------ holder shape
torch.Size([4, 23, 14, 221]) <------ holder wgt information


**I-NN-O**

Here I-NN-O means `Input - (Neural Network) --> Output`. 

In [26]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

info = info * holder_wgt.unsqueeze(-1)
print(info.shape, '<----- updated by holder_wgt')

torch.Size([4, 23, 14, 221]) <----- holder shape
torch.Size([4, 23, 14, 221, 128]) <----- info shape
torch.Size([4, 23, 14, 221, 128]) <----- updated by holder_wgt


## LLMEmbedding

### Module

In [27]:
import os
import torch
import numpy as np
# from ...utils.layerfn import orderSeq, restoreSeq
from fieldnn.utils.layerfn import orderSeq, restoreSeq
from transformers import AutoModel


class LLMEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 tokenizer, 
                 embedding_size, 
                 init, 
                 freeze = False):
        
        super(LLMEmbeddingLayer, self).__init__()
        
        self.tokenizer = tokenizer
        assert init == tokenizer.name_or_path 
        
        self.LLM = AutoModel.from_pretrained(init)
        self.hidden_size = self.LLM.config.hidden_size
        self.embedding_size = embedding_size
        
        self.linear  = torch.nn.Linear(self.hidden_size, self.embedding_size)
        self.init_linear_weights()
            
    def init_linear_weights(self):
        initrange = 0.1
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)
        
        
    def forward(self, holder):
        # 1. get leng_mask
        leng_mask = holder == 0
        
        # 2. get ordered holder
        ord_holder, ord_leng_mask, r_idx = self.reshape(holder, leng_mask)
        
        # 3. embedding ordered holder by LLM
        # expanding by the HuggingFace Language Model
        
        # 3.1 we might want to freeze LLM here
        # print(ord_holder.shape, '<--- ord_holder.shape')
        output = self.LLM(ord_holder)
        # 3.2 adjust the hidden dimension
        ord_info_output = output['last_hidden_state']
        ord_info_output = self.linear(ord_info_output) # bias might not be zeros.
        ord_info_output = ord_info_output.masked_fill(ord_leng_mask.unsqueeze(-1), 0)
    
        # 4. restore orderded output to original shape
        info = self.restore(ord_info_output, leng_mask, r_idx)
        
        return info
    
    
    def reshape(self, holder, leng_mask):
        nbs = np.array(holder.shape[:-1]).prod()
        ngrn = holder.shape[-1]
        # print(nbs, ngrn, dim)
        
        tmp_holder = holder.contiguous().view(nbs, ngrn)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_holder,    ord_leng, r_idx = orderSeq(tmp_holder, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_holder, ord_leng_mask, r_idx
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        

### Config

In [28]:
def get_llmembed_para(embed_size, tokenizer, init):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'tokenizer': tokenizer}
    return embed_para

### Usage

**recfldgrn information**

In [29]:
[i for i in batch_rfg]

['B-P@PatEcInfo-InfoGrn_wgt',
 'B-P@PatEcInfo-InfoGrn_tknidx',
 'B-P@PatEcInfo-InfoGrn_fldidx',
 'B-P-EC-PNSect@AllText-TknzGrn_wgt',
 'B-P-EC-PNSect@AllText-TknzGrn_tknidx',
 'B-P-EC-PNSect@AllText-TknzGrn_fldidx']

In [30]:
import torch

recfldgrn = 'P-EC-PNSect@AllText-TknzGrn'
full_recfldgrn = f'B-{recfldgrn}'

full_recfldgrn_tknidx = f'{full_recfldgrn}_tknidx'
info_idx = batch_rfg[full_recfldgrn_tknidx]
print(info_idx.shape)
holder = info_idx.long()

torch.Size([4, 23, 14, 221])


In [31]:
print(full_recfldgrn, '<-- full_recfldgrn')

recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn

print(recfldgrn, '<---- recfldgrn')
print(prefix_ids, '<---- prefix_ids')
print(rec, '<---- rec')
print(fld, '<---- fld')
print(grn, '<---- grn')

B-P-EC-PNSect@AllText-TknzGrn <-- full_recfldgrn
PNSect@AllText-TknzGrn <---- recfldgrn
['B', 'P', 'EC'] <---- prefix_ids
PNSect <---- rec
AllText <---- fld
TknzGrn <---- grn


In [32]:
import pandas as pd
# (2) get vocab information, in the `recfldgrn` task, we have already created the FldGrnInfo
fldgrn_folder = 'data/ProcData/FldGrnInfo/'
fullfldgrn_file = os.path.join(fldgrn_folder, full_recfldgrn[2:] + '.p')
print(fullfldgrn_file)
Info = pd.read_pickle(fullfldgrn_file)
Info

data/ProcData/FldGrnInfo/P-EC-PNSect@AllText-TknzGrn.p


RecLevel                                                          PNSect
RecLevelID                                                      PNSectID
SynFld                                                           AllText
GrnName                                                          TknzGrn
Rec2FldList_Dict       {'PNSect': ['SectName'], 'PNSectSent': ['Sente...
prefix_ids                                                   [PID, ECID]
focal_ids                                                     [PNSectID]
rec_folder                                       data/ProcData/RecFolder
recfldgrn                                         PNSect@AllText-TknzGrn
prefix_recfldgrn                             P-EC-PNSect@AllText-TknzGrn
grain_tkn_fn_string    def grain_tkn_fn(df_input):\n    # (0) Set up....
EmbedDict              {'tkn': {'embed_type': 'LLMEmbed', 'recfldgrn_...
use_wgt                                                            False
dtype: object

In [33]:
for sfx, para in Info['EmbedDict'].items():
    if para['embed_type'] == 'LLMEmbed':
        print(sfx)
        sfx, para
        break

tkn


In [34]:
vocab_size = para['vocab_size']
vocab_size

30522

In [35]:
tokenizer = para['tknz']

**get configuration**

In [36]:
init = tokenizer.name_or_path
embed_para = get_llmembed_para(embed_size, tokenizer, init)
embed_para

{'embedding_size': 128,
 'init': 'bert-base-uncased',
 'tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)}

**init NN model**

In [37]:
NN = LLMEmbeddingLayer(**embed_para)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**prepare input**

In [38]:
print(full_recfldgrn)

# (1) get the info_raw from batch_rfg
full_recfldgrn_sfx = full_recfldgrn + f'_{sfx}idx'
info_raw = batch_rfg[full_recfldgrn_sfx]
print(info_raw.shape, f'<---- info_raw for full_recfldgrn {sfx}: {full_recfldgrn_sfx}' )

full_recfldgrn_wgt = full_recfldgrn + f'_wgt'
info_wgt = batch_rfg[full_recfldgrn_wgt]
print(info_wgt.shape, f'<---- full_recfldgrn wgt: {full_recfldgrn_wgt}' )


holder = info_raw.long()
holder_wgt = info_raw.float() # torch.FloatTensor(info_raw)
    
print(holder.shape, '<------ holder shape')
print(holder_wgt.shape, '<------ holder wgt information')


B-P-EC-PNSect@AllText-TknzGrn
torch.Size([4, 23, 14, 221]) <---- info_raw for full_recfldgrn tkn: B-P-EC-PNSect@AllText-TknzGrn_tknidx
torch.Size([4, 23, 14, 221]) <---- full_recfldgrn wgt: B-P-EC-PNSect@AllText-TknzGrn_wgt
torch.Size([4, 23, 14, 221]) <------ holder shape
torch.Size([4, 23, 14, 221]) <------ holder wgt information


In [39]:
# holder[0,0,0]

In [40]:
# holder

**I-NN-O**

In [41]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

torch.Size([4, 23, 14, 221]) <----- holder shape
torch.Size([4, 23, 14, 221, 128]) <----- info shape


#  *Basic NN Type

## Module

In [20]:
import torch

# from .cateembed import CateEmbeddingLayer
# # from .numeembed import NumeEmbeddingLayer 
# from .llmembed import LLMEmbeddingLayer

class Expander_Layer(torch.nn.Module):
    
    def __init__(self, input_names_nnlvl, output_name_nnlvl, expander_para):
        super(Expander_Layer, self).__init__()
        
        
        # the input feature dim size and output feature dim size
        self.input_size = expander_para['input_size']
        self.output_size = expander_para['output_size']
        
        # input information
        assert len(input_names_nnlvl) == 1
        self.input_names_nnlvl = input_names_nnlvl
        self.input_name_nnlvl = input_names_nnlvl[0]
        
        # input with idx
        self.input_names_nnlvl_idx = [i for i in expander_para if self.input_name_nnlvl in i and 'idx' in i]
        
        # output information
        self.output_name_nnlvl = output_name_nnlvl
    
        # Part 1: NN
        self.EmbedDict = torch.nn.ModuleDict()
        for input_name_nnlvl_idx in self.input_names_nnlvl_idx:
            # for each input_name_nnlvl_idx, we assume we can find them in the INPUTS_TO_INFODICT
            assert 'Grn' in input_name_nnlvl_idx
            embed_para = expander_para[input_name_nnlvl_idx]
            
            nn_name, nn_para = embed_para['nn_name'], embed_para['nn_para']

            # input_name_nnlvl
            if nn_name.lower() == 'cateembed':
                Embed = CateEmbeddingLayer(**nn_para)
                self.EmbedDict[input_name_nnlvl_idx] = Embed
            elif nn_name.lower() == 'llmembed':
                Embed = LLMEmbeddingLayer(**nn_para)
                self.EmbedDict[input_name_nnlvl_idx] = Embed
            # elif nn_name.lower() == 'numeembed':
            #     # TODO: in the future
            else:
                raise ValueError(f'suffix is not correct "{self.input_name_nnlvl}"')

            self.embed_size = self.output_size
            
        self.use_wgt = expander_para['use_wgt']
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in expander_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.embed_size, **config)

    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        # names
        input_name_nnlvl = input_names_nnlvl[0]
        assert len(input_names_nnlvl) == 1
        assert input_name_nnlvl == self.input_name_nnlvl
    
        # get info_dict
        info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
        
        
        for input_name_nnlvl_idx in self.input_names_nnlvl_idx:
            assert input_name_nnlvl_idx in info_dict
        if self.use_wgt:
            assert (input_name_nnlvl + '_wgt') in info_dict

        # get embed
        embed_list = []
        for input_name_nnlvl_idx in self.input_names_nnlvl_idx:
            # INPUTS_TO_INFODICT is a part of batch_rfg
            holder = info_dict[input_name_nnlvl_idx]
            holder = holder.long()
            
            # print(holder.shape, f'<----- {input_name_nnlvl}')
            # leng_mask = holder == 0
            embed = self.EmbedDict[input_name_nnlvl_idx](holder)
            
            if self.use_wgt == True: 
                holder_wgt = info_dict[input_name_nnlvl + '_wgt']
                embed = embed * holder_wgt.unsqueeze(-1)
                
            # print(embed.shape, input_name_nnlvl)
            embed_list.append(embed)
            
        embed = torch.stack(embed_list, -2).mean(-2)
        
        for nn, layer in self.postprocess.items():
            embed = layer(embed)# masked_fill(leng_mask.unsqueeze(-1))
        
        return self.output_name_nnlvl, {'holder': holder, 'info': embed}
    

## Config

In [191]:
full_recfldgrn = 'B-P-EC-PNSect@AllText-TknzGrn'

In [192]:
import pandas as pd
# (2) get vocab information, in the `recfldgrn` task, we have already created the FldGrnInfo
fldgrn_folder = 'data/ProcData/FldGrnInfo/'
fullfldgrn_file = os.path.join(fldgrn_folder, full_recfldgrn[2:] + '.p')
print(fullfldgrn_file)
Info = pd.read_pickle(fullfldgrn_file)
EmbedDict = Info['EmbedDict']
[i for i in EmbedDict]

data/ProcData/FldGrnInfo/P-EC-PNSect@AllText-TknzGrn.p


['tkn', 'fld']

In [193]:
Info['use_wgt']

False

In [194]:
def get_cateembed_para(embed_size, vocab_size, init = 'random'):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'vocab_size': vocab_size}
    return embed_para


def get_llmembed_para(embed_size, tokenizer, init):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'tokenizer': tokenizer}
    return embed_para


def get_expander_para(full_recfldgrn, Info, embed_size, postprocess):
    
    expander_para = {}
    #(1) Input size, output size
    expander_para['input_size'] = None
    expander_para['output_size'] = embed_size
    expander_para['nn_type'] = 'expander'
    
    # (2) Loop Embed Info for each sfx
    EmbedDict = Info['EmbedDict']
    
    for sfx in EmbedDict:
        # embed_para = expander_para[input_name_nnlvl]
        input_name_nnlvl = full_recfldgrn + '_' + sfx + 'idx'
        
        EmbedConf = EmbedDict[sfx]
        nn_name = EmbedConf['embed_type']
        
        embed_para = {}
        embed_para['nn_name'] = nn_name
        
        if nn_name.lower() == 'cateembed':
            vocab_size = EmbedConf['vocab_size']
            init = 'random' # init = EmbedConf['init'] # TODO: to update in the future.
            para = get_cateembed_para(embed_size, vocab_size, init)
            embed_para['nn_para'] = para
        elif nn_name.lower() == 'llmembed':
            tokenizer = EmbedConf['tknz']
            init = tokenizer.name_or_path
            para = get_llmembed_para(embed_size, tokenizer, init)
            embed_para['nn_para'] = para
        else:
            raise ValueError(f'The NN "{nn_name}" is not available yet')
        
        expander_para[input_name_nnlvl] = embed_para
    
    # (3) Post Process
    expander_para['postprocess'] = postprocess
    
    
    # (4) use_wgt
    expander_para['use_wgt'] = Info.get('use_wgt', True)
    
    return expander_para

## Usage

In [195]:
import pandas as pd

full_recfldgrn = 'B-P-EC-PNSect@AllText-TknzGrn'
# (2) get vocab information, in the `recfldgrn` task, we have already created the FldGrnInfo
fldgrn_folder = 'data/ProcData/FldGrnInfo/'
fullfldgrn_file = os.path.join(fldgrn_folder, full_recfldgrn[2:] + '.p')
print(fullfldgrn_file)
Info = pd.read_pickle(fullfldgrn_file)
EmbedDict = Info['EmbedDict']
print([sfx for sfx in EmbedDict])

data/ProcData/FldGrnInfo/P-EC-PNSect@AllText-TknzGrn.p
['tkn', 'fld']


In [196]:
full_recfldgrn

'B-P-EC-PNSect@AllText-TknzGrn'

In [197]:
input_names_nnlvl = [full_recfldgrn]
input_names_nnlvl

['B-P-EC-PNSect@AllText-TknzGrn']

In [198]:
output_name_nnlvl = full_recfldgrn.replace('Grn', '')
output_name_nnlvl

'B-P-EC-PNSect@AllText-Tknz'

In [199]:
embed_size = 128
postprocess = {}

expander_para = get_expander_para(full_recfldgrn, Info, embed_size, postprocess)
expander_para

{'input_size': None,
 'output_size': 128,
 'nn_type': 'expander',
 'B-P-EC-PNSect@AllText-TknzGrn_tknidx': {'nn_name': 'LLMEmbed',
  'nn_para': {'embedding_size': 128,
   'init': 'bert-base-uncased',
   'tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)}},
 'B-P-EC-PNSect@AllText-TknzGrn_fldidx': {'nn_name': 'CateEmbed',
  'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 3}},
 'postprocess': {},
 'use_wgt': False}

In [200]:
input_names_nnlvl

['B-P-EC-PNSect@AllText-TknzGrn']

In [201]:
NN = Expander_Layer(input_names_nnlvl, output_name_nnlvl, expander_para)
# NN

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [202]:
# batch_rfg

INPUTS_TO_INFODICT = {}
INPUTS_TO_INFODICT[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
# INPUTS_TO_INFODICT

In [203]:
# INPUTS_TO_INFODICT = {k:{'holder': v} for k, v in batch_rfg.items() if full_recfldgrn in k and 'idx' in k}
# INPUTS_TO_INFODICT[f'{full_recfldgrn}_wgt'] = {'holder_wgt': batch_rfg[f'{full_recfldgrn}_wgt']}
# [i for i in INPUTS_TO_INFODICT]

In [204]:
output = NN(input_names_nnlvl, INPUTS_TO_INFODICT)

In [205]:
output_name_nnlvl, info_dict = output
# holder.shape

In [206]:
output_name_nnlvl

'B-P-EC-PNSect@AllText-Tknz'

In [207]:
holder = info_dict['holder']
print(holder.shape)

torch.Size([4, 23, 14, 221])


In [208]:
embed = info_dict['info']
print(embed.shape)

torch.Size([4, 23, 14, 221, 128])


In [209]:
[i for i in batch_rfg]

['B-P@PatEcInfo-InfoGrn_wgt',
 'B-P@PatEcInfo-InfoGrn_tknidx',
 'B-P@PatEcInfo-InfoGrn_fldidx',
 'B-P-EC-PNSect@AllText-TknzGrn_wgt',
 'B-P-EC-PNSect@AllText-TknzGrn_tknidx',
 'B-P-EC-PNSect@AllText-TknzGrn_fldidx']

In [210]:
# batch_rfg['B-P-EC-PNSect@AllText-TknzGrn_tknidx'][:, 0, 0, :]

In [211]:
embed[:, 0, 0, :, 0]

tensor([[-0.2139,  0.1847, -0.2423, -0.4179,  0.4112,  0.2471,  0.7898,  0.5596,
          0.7678,  0.5092,  0.6823,  1.0100,  0.5633,  0.1970,  1.0344,  0.4116,
          0.5255,  0.7157,  0.6562,  0.2446,  0.9512,  0.6135,  0.2989,  0.5979,
          0.5434,  0.4153,  0.1040,  0.8499,  0.6465,  0.2747,  0.6040,  0.1042,
          0.7627,  0.4801,  0.8360,  0.9201,  0.0298,  0.4416,  0.9291,  0.3996,
         -0.0069,  0.6184,  0.7762,  0.9895,  0.8839,  0.7529,  0.4055, -0.1771,
          0.2863,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [212]:
holder[:, 0, 0, :]

tensor([[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
   